In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder,OneHotEncoder


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sample_submission=pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
train['target'].value_counts()

In [ ]:
feats=train.drop(['id','target'],axis=1).columns
feats

In [ ]:
train.describe()

In [ ]:
folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [ ]:
X=train.drop('target',axis=1)

In [ ]:
le=LabelEncoder()
train['target']=le.fit_transform(train['target'])
y=train['target'].values

In [ ]:
# for i,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
#     print(f'Fold {i+1}')
#     print(f'Train index {len(trn_idx)} Valid index {len(val_idx)}')
#     trn_X,trn_Y=X.iloc[trn_idx,],y[trn_idx]
#     print(trn_X.shape,trn_Y.shape,trn_Y.reshape(-1,1).toarray())
#     val_X,val_Y=X.iloc[val_idx,],y[val_idx]
#     #print(trn_Y.value_counts())

In [ ]:
ohe=OneHotEncoder()

In [ ]:
oof_preds=np.zeros((train.shape[0],9))
sub=np.zeros((test.shape[0],9))
feature_importance=pd.DataFrame()
losses=[]
for i,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
    print(f'Fold {i+1}')
    trn_X,trn_Y=X[feats].iloc[trn_idx,],y[trn_idx]
    val_X,val_Y=X[feats].iloc[val_idx,],y[val_idx]
    clf=lgb.LGBMClassifier(
                    n_estimators=2000,
                    learning_rate=0.01,
                    num_leaves=30,
                    colsample_bytree=0.8,
                    subsample=0.8,
                    max_depth=7,
                    reg_alpha=0.1,
                    reg_lambda=0.1,
                    min_split_gain=0.01,
                    min_child_weight=2,
                    silent=-1,
                    verbose=-1
                    )
    clf.fit(trn_X,trn_Y,eval_set=[(trn_X,trn_Y),(val_X,val_Y)],
           eval_metric='logloss',verbose=100,early_stopping_rounds=30)
    preds=clf.predict_proba(val_X,num_iteration=clf.best_iteration_)
    oof_preds[val_idx]=clf.predict_proba(val_X,num_iteration=clf.best_iteration_)
    loss=log_loss(ohe.fit_transform(val_Y.reshape(-1,1)).toarray(),preds)
    sub+=clf.predict_proba(test[feats],num_iteration=clf.best_iteration_)/folds.n_splits
    fold_importance_df=pd.DataFrame()
    fold_importance_df['features']=feats
    fold_importance_df['importance']=clf.feature_importances_
    fold_importance_df['folds']=i+1
    feature_importance=pd.concat([feature_importance,fold_importance_df],axis=0)
    
    loss=log_loss(val_Y,oof_preds[val_idx])
    
    print(f'Loss for fold {i+1} {loss}')
    losses.append(loss)      
    
print(f'Mean loss {np.mean(losses)}')

In [ ]:
sub.shape

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.iloc[:,1:]=sub

In [ ]:
sample_submission.to_csv('sample_submission.csv',index=False)